In [1]:
import MDAnalysis
import MDAnalysis.analysis.hbonds
import pandas as pd
import numpy as np

from collections import defaultdict 
import networkx as nx
import matplotlib.pyplot as plt
import sys
sys.setrecursionlimit(1000)
print(sys.getrecursionlimit())

/Users/zhangyingying/anaconda/lib/python3.5/site-packages/MDAnalysis/analysis/hbonds/hbond_analysis.py:346: DeprecationWarning: This module is deprecated as of MDAnalysis version 1.0.It will be removed in MDAnalysis version 2.0Please use MDAnalysis.analysis.hydrogenbonds.hbond_analysis instead.
  category=DeprecationWarning


1000


## Function for graph and finding paths

In [43]:
def addEdge(graph,u,v): 
    graph[u].append(v) 
def find_all_path(graph, start, path, paths):
    if len(path) == 6:
        return paths.append(list(path))
    if len(graph[start]) == 0:
        return paths.append(list(path))

    for node in graph[start]:
        if node in path:
                continue
        path.append(node)
        find_all_path(graph, node, path, paths)
        path.pop()

## Loading the pdb file or dcd file with psf

In [2]:
DCD = '/Users/zhangyingying/Dropbox (City College)/Yingying/large_file/new_trajectories_PSII_wt/step7_50.dcd'
PDB = '/Users/zhangyingying/Dropbox (City College)/Yingying/large_file/new_trajectories_PSII_wt/frame50_56-stripped.pdb'
PSF = '/Users/zhangyingying/Dropbox (City College)/Yingying/large_file/new_trajectories_PSII_wt/step5_charmm2omm_keep.psf'

## Get chain name for each atom
### For some cases, there are duplicate resname+resid but with different chain name, for distinguishing these different residues, we need to know the chain name

In [3]:
chain = {}
i = 0
pdb = open(PDB, 'r')
for line in pdb:
    if line[0:4] != 'ATOM':
        continue
    chain[i] = line[21:22]
    i += 1
print(chain)

## Calculate angles and distances for atoms and filter HBs

In [3]:
u = MDAnalysis.Universe(PDB)
h3 = MDAnalysis.analysis.hbonds.HydrogenBondAnalysis(u, 'not resname ALA and not resname GLN and not resname GLY and not resname ILE and not resname LEU and not resname PHE and not resname PRO and not resname VAL', 
                                                          'not resname ALA and not resname GLN and not resname GLY and not resname ILE and not resname LEU and not resname PHE and not resname PRO and not resname VAL', distance=3.5, angle=90.0, acceptors = {'O1', 'O2'})
h3.run()
h3.generate_table()
df3 = pd.DataFrame.from_records(h3.table)
h3.generate_table() 
df3 = pd.DataFrame.from_records(h3.table)
print(df3.head(10))

/Users/zhangyingying/anaconda/lib/python3.5/site-packages/MDAnalysis/topology/guessers.py:80: UserWarning: Failed to guess the mass for the following atom types: M
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/Users/zhangyingying/anaconda/lib/python3.5/site-packages/MDAnalysis/topology/PDBParser.py:330: UserWarning: Element information is absent or missing for a few atoms. Elements attributes will not be populated.
  warnings.warn("Element information is absent or missing for a few "
/Users/zhangyingying/anaconda/lib/python3.5/site-packages/MDAnalysis/analysis/hbonds/hbond_analysis.py:580: DeprecationWarning: This class is deprecated as of MDAnalysis version 1.0 and will be removed in version 2.0.Please use MDAnalysis.analysis.hydrogenbonds.hbond_analysis.HydrogenBondAnalysis instead.
  category=DeprecationWarning
/Users/zhangyingying/anaconda/lib/python3.5/site-packages/MDAnalysis/coordinates/base.py:865: UserWarning: Reader has no dt 

   time  donor_index  acceptor_index donor_resnm  donor_resid donor_atom  \
0   0.0           13              20         ASN           12          H   
1   0.0           13          127926         ASN           12          H   
2   0.0           46          127950         TRP           14          H   
3   0.0           56           28536         TRP           14        HE1   
4   0.0           70              25         GLU           15          H   
5   0.0           85              25         ARG           16          H   
6   0.0          129              68         CYS           18          H   
7   0.0          129              83         CYS           18          H   
8   0.0          129             135         CYS           18          H   
9   0.0          136              68         CYS           18         HG   

  acceptor_resnm  acceptor_resid acceptor_atom  distance       angle  
0            ASN              12           OD1  3.309775   94.632241  
1           TIP3     

In [4]:
df3.to_csv('/Users/zhangyingying/Dropbox (City College)/Yingying/PSII/quinone/hb_network/1000ns_connection_his252sele.csv')

## Give chain names for protein and index for water molecules

In [23]:
index_donor = []
index_accept = []
for index2, row2 in df3.iterrows():
    if row2['donor_resnm'] == 'TIP3'and row2['acceptor_resnm'] != 'TIP3':
        if row2['donor_atom'] == 'H1':
            index_donor.append(row2['donor_resnm'] + '_' + str(row2['donor_index']-1))
            index_accept.append(row2['acceptor_resnm'] + '_' + chain[row2['acceptor_index']] + '_' + str(row2['acceptor_resid']))
        if row2['donor_atom'] == 'H2':
            index_donor.append(row2['donor_resnm'] + '_' + str(row2['donor_index']-2))
            index_accept.append(row2['acceptor_resnm'] + '_' + chain[row2['acceptor_index']] + '_' + str(row2['acceptor_resid'])) 

    elif row2['acceptor_resnm'] == 'TIP3' and row2['donor_resnm'] != 'TIP3':
        index_accept.append(row2['acceptor_resnm'] + '_' + str(row2['acceptor_index']))
        index_donor.append(row2['donor_resnm'] + '_' + chain[row2['donor_index']] + '_' + str(row2['donor_resid']))
    elif row2['acceptor_resnm'] == 'TIP3' and row2['donor_resnm'] == 'TIP3':
        if row2['donor_atom'] == 'H1':
            index_donor.append(row2['donor_resnm'] + '_' + str(row2['donor_index']-1))
            index_accept.append(row2['acceptor_resnm'] + '_' + str(row2['acceptor_index']))
        if row2['donor_atom'] == 'H2':
            index_donor.append(row2['donor_resnm'] + '_' + str(row2['donor_index']-2))
            index_accept.append(row2['acceptor_resnm'] + '_' + str(row2['acceptor_index']))
    else:
        index_donor.append(row2['donor_resnm'] + '_' + chain[row2['donor_index']] + '_' + str(row2['donor_resid']))
        index_accept.append(row2['acceptor_resnm'] + '_' + chain[row2['acceptor_index']] + '_' + str(row2['acceptor_resid']))
df3['donor_residue'] = index_donor
df3['acceptor_residue'] = index_accept
print(df3.head(10))

   time  donor_index  acceptor_index donor_resnm  donor_resid donor_atom  \
0   0.0           13              20         ASN           12          H   
1   0.0           13          127926         ASN           12          H   
2   0.0           46          127950         TRP           14          H   
3   0.0           56           28536         TRP           14        HE1   
4   0.0           70              25         GLU           15          H   
5   0.0           85              25         ARG           16          H   
6   0.0          129              68         CYS           18          H   
7   0.0          129              83         CYS           18          H   
8   0.0          129             135         CYS           18          H   
9   0.0          136              68         CYS           18         HG   

  acceptor_resnm  acceptor_resid acceptor_atom  distance       angle  \
0            ASN              12           OD1  3.309775   94.632241   
1           TIP3   

## Filter the sidechain hydrogen bond (hide the backbone HBs)

In [25]:
hb = pd.DataFrame()
dic_hdonnor = {'ASP':['HD1', 'HD2'], 'ARG': ['HH11', 'HH12', 'HH21', 'HH22', 'HE'], 'GLU':['HE1', 'HE2'], 'HIS':['HD1', 'HE2'], 'HSD':['HD1', 'HE2'], 'HSE':['HD1', 'HE2'], 'HSP':['HD1', 'HE2'],
                'SER':['HG'], 'THR':['HG1'], 'ASN':['HD21', 'HD22'], 'GLN':['HE21', 'HE22'], 'CYS':['HG'], 'TYR':['HH'], 'TRP':['HE1'], 'LYS':['HZ1', 'HZ2', 'HZ3'], 'TIP3':['H1', 'H2'], 'HOH':['1H', '2H']}
dic_accept = {'ASP':['OD1', 'OD2'], 'HCO': ['OC1', 'OC2'], 'ARG': ['NE', 'NH1', 'NH2'], 'GLU':['OE1', 'OE2'], 'HSD':['ND1', 'NE2'], 'HSE':['ND1', 'NE2'], 'HSP':['ND1', 'NE2'], 'HIS':['ND1', 'NE2'],
                'SER':['OG'], 'THR':['OG1'], 'ASN':['OD1'], 'GLN':['OE1'], 'CYS':['SG'], 'TYR':['OH'], 'LYS':['NZ'], 'MET':['SD'], 'CLX':['CLX'], 'CLA':['CLA'], 'OX2':['OX2'], 'PL9':['O1', 'O2'], 'FX':['FX'], 'TIP3':['OH2'], 'HOH':['O'], 'MQ8':['O1', 'O2']}
donor_residue_pick = []
acceptor_residue_pick = []
donor_atom_pick = []
acceptor_atom_pick = []
for index, row in df3.iterrows():
    if row['donor_resnm'] in dic_hdonnor.keys() and row['acceptor_resnm'] in dic_accept.keys():
        if row['donor_atom'] in dic_hdonnor[row['donor_resnm']] and row['acceptor_atom'] in dic_accept[row['acceptor_resnm']]:
            donor_residue_pick.append(row['donor_residue'])
            acceptor_residue_pick.append(row['acceptor_residue'])
            donor_atom_pick.append(row['donor_atom'])
            acceptor_atom_pick.append(row['acceptor_atom'])
    else:
        continue
# all connection network
hb_two = pd.DataFrame({'donor_residue':donor_residue_pick, 'donor_atom':donor_atom_pick, 'acceptor_residue':acceptor_residue_pick, 'acceptor_atom':acceptor_atom_pick})
print(hb_two.head(10))

  acceptor_atom acceptor_residue donor_atom donor_residue
0            OG         SER_H_25        HE1      TRP_A_14
1           OD1         ASN_A_26        HE1      TRP_A_20
2           OH2       TIP3_47469       HD21      ASN_A_26
3           OH2      TIP3_127254       HH11      ARG_A_27
4           OH2      TIP3_127353       HH11      ARG_A_27
5           OH2      TIP3_127254       HH12      ARG_A_27
6           OH2      TIP3_128304       HH12      ARG_A_27
7           OH2      TIP3_128304       HH22      ARG_A_27
8           OE1        GLU_A_132         HH      TYR_A_29
9           OE2        GLU_A_132         HH      TYR_A_29


## Divide all connections to two groups: directly connection, connection via water molecules

In [26]:
donor_residue = []
acceptor_residue = []
donor_residue2 = []
acceptor_residue2 = []
for row in range(len(hb_two)):
    if hb_two['donor_residue'][row][0:3] != 'TIP' and hb_two['acceptor_residue'][row][0:3] != 'TIP': 
        if hb_two['donor_residue'][row] == hb_two['acceptor_residue'][row]:
            continue
        else:
            donor_residue.append(hb_two['donor_residue'][row])
            acceptor_residue.append(hb_two['acceptor_residue'][row])
    else:
        if hb_two['donor_residue'][row] == hb_two['acceptor_residue'][row]:
            continue
        else:
            donor_residue2.append(hb_two['donor_residue'][row])
            acceptor_residue2.append(hb_two['acceptor_residue'][row])
dire_con = pd.DataFrame({'donor_residue': donor_residue, 'acceptor_residue': acceptor_residue, 'wat_num': [0]*len(donor_residue)})
wat_con = pd.DataFrame({'donor_residue': donor_residue2, 'acceptor_residue': acceptor_residue2})

# connection via water
wat_con = wat_con.drop_duplicates()
wat_con.index = range(0, len(wat_con))

# direct connection
dire_con = dire_con.drop_duplicates()
dire_con.index = range(0, len(dire_con))
print('Direct connection:', len(dire_con))
print('Connection with water:', len(wat_con))

Direct connection: 303
Connection with water: 2691


## Build graph for the connction via water

In [39]:
graph = defaultdict(list)
for i in range(len(wat_con)):
    addEdge(graph, wat_con['donor_residue'][i], wat_con['acceptor_residue'][i])
print(graph)
# print(graph['TIP3_127788'])

defaultdict(<class 'list'>, {'TIP3_128523': ['TIP3_127788', 'TIP3_129585'], 'TIP3_129054': ['TIP3_129312', 'TIP3_130125'], 'TIP3_129462': ['ASP_M_169', 'TIP3_128106', 'TIP3_129702', 'TIP3_128775', 'TIP3_129048'], 'TIP3_128214': ['TIP3_127143', 'TIP3_127917', 'ASP_B_276'], 'SER_M_221': ['TIP3_130044'], 'TIP3_127257': ['TIP3_127887', 'TIP3_127836'], 'TIP3_128514': ['TIP3_129528'], 'LYS_B_373': ['TIP3_128022'], 'TIP3_47751': ['THR_C_158'], 'TIP3_47754': ['TIP3_47733'], 'ARG_A_129': ['TIP3_47466'], 'TIP3_47490': ['ASP_A_308', 'TIP3_129336'], 'ARG_P_31': ['TIP3_128895', 'TIP3_129561'], 'TIP3_128997': ['ASP_M_205', 'GLU_M_210'], 'ASN_A_303': ['TIP3_47538', 'TIP3_47586'], 'TIP3_47550': ['ASP_A_61', 'TIP3_47517', 'TIP3_47574'], 'SER_B_391': ['TIP3_129015'], 'TIP3_129147': ['TIP3_127164', 'TIP3_129192', 'TIP3_128190', 'TIP3_129069', 'TIP3_129270'], 'HIS_M_228': ['TIP3_128223', 'TIP3_130044'], 'TIP3_129765': ['TIP3_129846'], 'TIP3_129216': ['GLU_C_413'], 'TIP3_128037': ['TIP3_127794', 'TIP3_1281

## Find all the paths in the graph

In [45]:
visited = []
path = []
for res in range(len(wat_con)):
    results = []
    if wat_con['donor_residue'][res] not in visited and wat_con['donor_residue'][res][0:3] != 'TIP':
        find_all_path(graph, wat_con['donor_residue'][res], [wat_con['donor_residue'][res]], results)
        path = path + results
        visited.append(wat_con['donor_residue'][res])
    else:
        continue
print(path[0:4])

[['ASN_A_26', 'TIP3_47469'], ['ARG_A_27', 'TIP3_127254', 'TIP3_128304', 'ASP_C_473'], ['ARG_A_27', 'TIP3_127254', 'TIP3_128304', 'TIP3_129822', 'ASP_C_473'], ['ARG_A_27', 'TIP3_127254', 'TIP3_128304', 'TIP3_129822', 'TIP3_127497', 'TIP3_128358']]


## Count the water number between residues

In [48]:
donor = []
accept = []
wat_num = []
for item in path:
    donor_column = [item[0]]
    accpt_column = []
    count = 0
    for r in range(1, len(item)):
        if item[r][0:3] != 'TIP':
            donor_column.append(item[r])
            accpt_column.append(item[r])
            wat_num.append(count)
            count = 0
        else:

            count += 1
    if len(donor_column) > len(accpt_column):
        donor_column.pop()
    else:
        accpt_column.pop()
    donor.extend(donor_column)
    accept.extend(accpt_column)
print(donor[0], accept[0], wat_num[0])

ARG_A_27 ASP_C_473 2


## Put all data in dataframe and count the frequency of the connection

In [49]:
direct_connection = pd.DataFrame(columns = ['donor_residue', 'acceptor_residue'])
one_water_connection = pd.DataFrame(columns = ['donor_residue', 'acceptor_residue'])
two_water_connection = pd.DataFrame(columns = ['donor_residue', 'acceptor_residue'])
three_water_connection = pd.DataFrame(columns = ['donor_residue', 'acceptor_residue'])
four_water_connection = pd.DataFrame(columns = ['donor_residue', 'acceptor_residue'])
visited_1 = [] 
visited_2 = []
visited_3 = []
visited_4 = []

In [50]:
res_wat_res = pd.DataFrame({'donor_residue': donor, 'acceptor_residue': accept, 'wat_num': wat_num})
res_wat_res = res_wat_res.drop_duplicates()
hb_network = pd.concat([dire_con, res_wat_res])
hb_network.index = range(0, len(hb_network))
for i in range(0, len(hb_network)):
    if hb_network['wat_num'][i] == 0:
        new_row = pd.Series({'donor_residue': hb_network['donor_residue'][i], 'acceptor_residue': hb_network['acceptor_residue'][i]})
        direct_connection = direct_connection.append(new_row, ignore_index=True)

    if hb_network['wat_num'][i] <= 1 and [hb_network['donor_residue'][i], hb_network['acceptor_residue'][i]] not in visited_1:
        visited_1.append([hb_network['donor_residue'][i], hb_network['acceptor_residue'][i]])
        new_row = pd.Series({'donor_residue': hb_network['donor_residue'][i], 'acceptor_residue': hb_network['acceptor_residue'][i]})
        one_water_connection = one_water_connection.append(new_row, ignore_index=True)

    if hb_network['wat_num'][i] <= 2 and [hb_network['donor_residue'][i], hb_network['acceptor_residue'][i]] not in visited_2:
        visited_2.append([hb_network['donor_residue'][i], hb_network['acceptor_residue'][i]])
        new_row = pd.Series({'donor_residue': hb_network['donor_residue'][i], 'acceptor_residue': hb_network['acceptor_residue'][i]})
        two_water_connection = two_water_connection.append(new_row, ignore_index=True)

    if hb_network['wat_num'][i] <= 3 and [hb_network['donor_residue'][i], hb_network['acceptor_residue'][i]] not in visited_3:
        visited_3.append([hb_network['donor_residue'][i], hb_network['acceptor_residue'][i]])
        new_row = pd.Series({'donor_residue': hb_network['donor_residue'][i], 'acceptor_residue': hb_network['acceptor_residue'][i]})
        three_water_connection = three_water_connection.append(new_row, ignore_index=True)

    if hb_network['wat_num'][i] <= 4 and [hb_network['donor_residue'][i], hb_network['acceptor_residue'][i]] not in visited_4:
        visited_4.append([hb_network['donor_residue'][i], hb_network['acceptor_residue'][i]])
        new_row = pd.Series({'donor_residue': hb_network['donor_residue'][i], 'acceptor_residue': hb_network['acceptor_residue'][i]})
        four_water_connection = four_water_connection.append(new_row, ignore_index=True)
print(direct_connection.head(5))

  donor_residue acceptor_residue
0      TRP_A_14         SER_H_25
1      TRP_A_20         ASN_A_26
2      TYR_A_29        GLU_A_132
3      ARG_A_64         THR_M_75
4      ASN_A_75         SER_A_68


## If we have more than one frame, we need to append all connection in one dataFrame

In [51]:
Direct = pd.DataFrame(columns = ['donor_residue', 'acceptor_residue'])
One_water = pd.DataFrame(columns = ['donor_residue', 'acceptor_residue'])
Two_water = pd.DataFrame(columns = ['donor_residue', 'acceptor_residue'])
Three_water = pd.DataFrame(columns = ['donor_residue', 'acceptor_residue'])
Four_water = pd.DataFrame(columns = ['donor_residue', 'acceptor_residue'])
# if we need to calculate more than one frame
Direct = pd.concat([Direct, direct_connection])
One_water = pd.concat([One_water, one_water_connection])
Two_water = pd.concat([Two_water, two_water_connection])
Three_water = pd.concat([Three_water, three_water_connection])
Four_water = pd.concat([Four_water, four_water_connection])
# calculate the frequency for all the connections
Direct = Direct.groupby(['donor_residue', 'acceptor_residue']).size().reset_index(name='Frequency')
One_water = One_water.groupby(['donor_residue', 'acceptor_residue']).size().reset_index(name='Frequency')
Two_water = Two_water.groupby(['donor_residue', 'acceptor_residue']).size().reset_index(name='Frequency')
Three_water = Three_water.groupby(['donor_residue', 'acceptor_residue']).size().reset_index(name='Frequency')
Four_water = Four_water.groupby(['donor_residue', 'acceptor_residue']).size().reset_index(name='Frequency')
print(Direct.head(5))

  donor_residue acceptor_residue  Frequency
0     ARG_A_136         ASP_H_27          1
1     ARG_A_136        GLU_A_132          1
2     ARG_A_136         TYR_A_29          1
3     ARG_A_140        GLU_D_219          1
4     ARG_A_257         ASP_D_25          1
